#### Import third-party libraries needed to run calculations

In [1]:
import pandas as pd
import numpy as np

#### Provide assumptions applicable to all energy generation technologies
These are the assumptions applicable to all energy generation technologies in the Lazard report

In [2]:
perc_debt = .6
discount_rate = .08
o_and_m_growth_rate = .0225
irr = .12

#### Provide configuration specific to each energy generation technology

There are two configs for wind: One based on [Lazard's 2019 LCOE report](https://www.lazard.com/media/451086/lazards-levelized-cost-of-energy-version-130-vf.pdf) and another based on [the Corporate Finance Institute's](https://corporatefinanceinstitute.com/resources/knowledge/finance/levelized-cost-of-energy-lcoe/) spreadsheet which used Lazard's 2018 report as a reference

**Notes** 
* Only (onshore) wind is provided here. 
* The config options for **Lazard** are more comprehensive, and therefore more complicated. Specifically, it allows you to calculate a range of LCOE. 
* The **CFI** config simplifies this to provide one LCOE estimate instead of a range, and does not include plant capacity, capacity factor, O&M costs per MWh, and initial investment per kW. Instead, it provides annual electricity generation of the plant, total initial investment, and total O&M costs.
* The shared config options between **CFI** and **Lazard** include `construction_time` (in months), `facility_life` (in years), `fuel_cost` (per year), and `o_and_m_growth_rate` (per year)

In [3]:
wind_config_lazard = {
    "capacity_factor_low" : .38,      # perc: 0 to 1
    "capacity_factor_high" : .55,     # perc: 0 to 1
    "construction_time" : 12,         # months
    "facility_life" : 20,             # years
    "fuel_cost" : 0,                  # float
    "o_and_m_low" : 28.,              # $/MWh
    "o_and_m_high" : 36.5,            # $/MWh
    "o_and_m_growth_rate" : .0225,    # perc: 0 to 1
    "plant_capacity" : 150,           # MW
    "total_capital_cost_low" : 1100,  # $/kW
    "total_capital_cost_high" : 1500, # $/kW
}

wind_config_cfi = {
    "annual_elec_output" : 3000.,     # in kWh
    "construction_time" : 12,         # months
    "facility_life" : 10,             # years
    "fuel_cost" : 0,                  # float
    "o_and_m_costs" : 100.,           # in $1,000
    "o_and_m_growth_rate" : .02,      # perc: 0 to 1
    "init_investment" : 1500,         # in $1000s
}



## Simplified LCOE Calc

In [6]:
def lcoe_calc_simplified(config):
    
    construction_time_in_months = config['construction_time']
    init_investment = config['init_investment']
    annual_fuel_costs = config['fuel_cost']
    annual_elec_output = config['annual_elec_output']
    o_and_m_costs = config['o_and_m_costs']
    o_and_m_growth_rate = config['o_and_m_growth_rate']
    project_lifetime_in_yrs = config['facility_life']
    
    total_npv_costs = init_investment
    total_npv_output = 0
    
    construction_time_in_yrs = float(construction_time_in_months) / float(12)
    
    for i in np.arange(construction_time_in_yrs, project_lifetime_in_yrs):
        years_since_proj_start = i + 1 # python index starts at 0, increment by 1 to make calcs for each subsequent yr
        years_operational = i + 1 - construction_time_in_yrs 
        accum_discount_rate = 1. / ((1 + float(discount_rate)) ** years_since_proj_start) 

        #### Start: COSTS s ####
        annual_o_and_m_costs = o_and_m_costs + annual_fuel_costs \
            if years_operational == 1 \
            else (1 + o_and_m_growth_rate) * annual_o_and_m_costs

    #     if years_operational == 1: # first year operational
    #         annual_o_and_m_costs = o_and_m_costs + annual_fuel_costs
    #  
    #     else : # each subsequent yr
    #         annual_o_and_m_costs = (1 + o_and_m_growth_rate) * annual_o_and_m_costs

        annual_o_and_m_costs_w_fuel = annual_o_and_m_costs + annual_fuel_costs
        annual_npv_costs = annual_o_and_m_costs_w_fuel * accum_discount_rate
        total_npv_costs += annual_npv_costs
        #### End: COSTS ####

        #### Start: OUTPUT ####
        annual_npv_output = (annual_elec_output * accum_discount_rate)
        total_npv_output += annual_npv_output
        
    lcoe_float = total_npv_costs / total_npv_output 
    lcoe_string = '${:,.2f}/kWh'.format(lcoe_float)
    
    return_obj = {
        "npv_costs" : total_npv_costs,
        "npv_output" : total_npv_output,
        "lcoe" : lcoe_float,
        "lcoe_string" : lcoe_string
    }
    return return_obj

In [7]:
lcoe_obj = lcoe_calc_simplified(wind_config_cfi)

facility_lifespan = wind_config_cfi['facility_life']
annual_elec_gen = wind_config_cfi['annual_elec_output']
total_npv_costs = '${:,.2f}'.format(lcoe_obj['npv_costs'])
total_npv_output = '${:,.2f}'.format(lcoe_obj['npv_output'])
lcoe_per_kwh = lcoe_obj['lcoe_string']

summary_string = """
    Over the {facility_lifespan} year lifespan of the plant producing {annual_elec_gen} kWH per year once operational, 
the total NPV of costs was {total_npv_costs} while the NPV of revenue from electricity generation was {total_npv_output}.
    
    Therefore, the LCOE of this hypothetical plant is {lcoe_per_kwh}.
""".format(**locals())
print(summary_string)


    Over the 10 year lifespan of the plant producing 3000.0 kWH per year once operational, 
the total NPV of costs was $2,120.61 while the NPV of revenue from electricity generation was $17,352.47.
    
    Therefore, the LCOE of this hypothetical plant is $0.12/kWh.



## Full LCOE Calc

In [ ]:
def lcoe_calc_simplified(config):
    
    return lcoe